# Building Autoencoders in Keras

## What are auto encoders?

Auto encoders are used as compression and decompression algorithms which are learned from data instead of engineered.

In [ ]:
from IPython.display import Image
Image('https://blog.keras.io/img/ae/autoencoder_schema.jpg')

# Let's build the simplest autoencoder possible

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

Let's create a separate encoder model

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

And a decoder model

In [ ]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

We will use a per-pixel binary crossentropy loss, which is generally used for binary classification.

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Load the MNIST data set, and discard the labels since we're only interested in encoding/decoding the images.

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

Show an example of an image in the data set

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train[5, :, :])

Normalize the training data to values between 0 and 1, and flatten the images into vectors of size 784.

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

Lets train the encoder for 50 epochs (50 sweeps over the complete training set.)

In [ ]:
history = autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True,
                          validation_data=(x_test, x_test))

Make a plot of the convergence history of the training 

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'], label="Training loss")
plt.plot(history.history['val_loss'], label="Validation loss")
plt.grid("on")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

The training loss and validation loss are starting to flatten out, indicating that the model is close to convergence. We can visualize the reconstructed inputs with the encoder and decoder network.

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

As shown in the figure, we are losing quite some detail with this basic approach.

In [ ]:
def plot_digits(n, x_test, decoded_imgs):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

plot_digits(n=10, x_test=x_test, decoded_imgs=decoded_imgs)

# Deep autoencoder

We do not have to limit ourselves to single layers as encoders and decoders. We can add more layers as follows.

Try to change number of hidden units, and the number of layers to see how the architecture changes. Use `model.summary()` to print the shape of the input and outputs of each layer.

In [ ]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

Lets train the new model. We are now training for 100 epochs which results in a longer training time.

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

history = autoencoder.fit(x_train, x_train, epochs=100, batch_size=256, shuffle=True,
                          validation_data=(x_test, x_test))

Review the convergence history again to make sure that the model is converged and is not overfitting the data.

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'], label="Training loss")
plt.plot(history.history['val_loss'], label="Validation loss")
plt.grid("on")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

The final loss is around 0.097, slightly better than our previous approach. Let's review the images.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

plot_digits(n=10, x_test=x_test, decoded_imgs=decoded_imgs)

# Application to image denoising

Let's train a new network which is able to remove noise from images. We simply add artificial noise to the training images.

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

Let's review the noisy images.

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Can our encoder remove this noise?

In [ ]:
input_img = Input(shape=(28, 28, 1))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Training again.

In [ ]:
autoencoder.fit(x_train_noisy, x_train, epochs=2, batch_size=128, shuffle=True, validation_data=(x_test_noisy, x_test))

Finally review the result.

In [ ]:
decoded_imgs = autoencoder.predict(x_test_noisy)

plot_digits(n=10, x_test=x_test_noisy, decoded_imgs=decoded_imgs)